## this notebook was developed on Google Colab


### dependencies

In [ ]:
!pip install -qU langchain langchain_openai langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00


In [ ]:
!pip install -qU --disable-pip-version-check qdrant-client pymupdf tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.0 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.0 which is incompatible.
google-api-core 2.11.1 requires protobuf!=3.20.0,

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

### rag chain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

You are a helpful assistant. Use the available context to answer the question. If you can't answer the question, say you don't know.
"""

chain_prompt = ChatPromptTemplate.from_template(PROMPT)

In [ ]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser

llm_chain = (
    {"question": itemgetter("question")}
    | chain_prompt | openai_chat_model | StrOutputParser()
)

In [ ]:
llm_chain.invoke({"question" : "What does the 'context' in 'long context' refer to?"})

"In the context of 'long context' in the provided text, the term 'context' refers to a coherent text, such as a book or a long paper, that is being used as the basis for generating question-answer pairs and analyzing information."

In [ ]:
paper_title = llm_chain.invoke({"question" : "Retrieve the paper title as string without any adittional word."})
paper_title

'Extending Llama-3’s Context'

### tools

In [ ]:
from typing import Annotated, List, Tuple, Union

def write_csv_file(content: Annotated[str, "The string final content to write to the file."], filename: Annotated[str, "The filename to save the file as, must be the dataset name. do NOT include .csv"]):
  """Writes the given content as a csv file to the local directory.

  Args:
    content: The string content to write to the file.
    filename: The filename to save the file as, must be the dataset name. do NOT include .csv
  """
  with open(f"{filename}.csv", "w") as f:
    f.write(content)

In [ ]:
def retrieve_paper_title():
  """Returns the paper title to use later"""
  return rag_chain.invoke({"question" : "Retrieve the paper title as string without any adittional word."})

In [ ]:
def retrieve_information(query: Annotated[str, "query to ask the retrieve information tool"], paper_title: Annotated[str, 'title of the paper']):
    f"""Use Retrieval Augmented Generation to retrieve information about {paper_title} paper."""
    return rag_chain.invoke({"question" : query})

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools, Tool
from langchain_openai import ChatOpenAI
from langchain.tools import StructuredTool

retrieve_paper_title = StructuredTool.from_function(
                func=retrieve_paper_title,
                name='RetrievePaperTitle',
                description="retrieve the paper's title")

retrieve_information = StructuredTool.from_function(
                func= retrieve_information,
                name= 'RetrieveInformation',
                description= 'search for information about the given paper using Retrieval Augmented Generation.',)

write_content = StructuredTool.from_function(
                func=write_markdown_file,
                name="WriteContent",
                description="write the final content as a markdown file to the local directory.",)


tools = [retrieve_paper_title, retrieve_information, write_content]

In [ ]:
llm = ChatOpenAI(temperature=0.0)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the retrieve_information tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)#, handle_parsing_errors=True)

In [ ]:
output = agent_executor.invoke(
    {
      "input": """Identify and answer the following questions based on the paper content and reasearch:
        -What is the research problem the paper attempts to address? What is the motivation?
        -What are the claimed contributions and novelties of the paper?
        -How do the authors substantiate their claims (methodology, experiments, proofs etc.)?
        -What are the main conclusions and lessons learned?
        -Is the research problem significant and not artificial?
        -Are the claimed contributions really significant and novel compared to existing work?
        -Are the claims and arguments valid, or are there flaws in the methodology, proofs, experiments etc.?
        -What is the core of the research problem? Are there alternative approaches?
        -Are there different ways to substantiate or argue against the claims?
        -Can the results be strengthened or applied to other contexts?
        -What are the open problems raised that could lead to further research?

        Then, based on the questions and answers, use and complete the following template to write the final content to the disk using the paper's title as filename:

        ## ReplaceThisWithPaperTitle

        ### Comprehension:
        \n- **Research Problem and Motivation:**
        \n- **Claimed Contributions and Novelties:**
        \n- **Substantiation of Claims:**
        \n- **Main Conclusions and Lessons Learned:**

        ### Evaluation:
        \n- **Significance of Research Problem:**
        \n- **Significance and Novelty of Contributions:**
        \n- **Validity of Claims and Arguments:**

        ### Synthesis:
        \n- **Core Research Problem:**
        \n- **Alternative Approaches and Substantiation:**
        \n- **Strengthening and Application of Results:**
        \n- **Open Problems for Further Research:**
        """,
    }
)



> Entering new AgentExecutor chain...

Invoking: `RetrievePaperTitle` with `{}`


Extending Llama-3’s Context
Invoking: `RetrieveInformation` with `{'query': 'research problem and motivation of the paper', 'paper_title': 'Extending Llama-3’s Context'}`


The research problem addressed in the paper is the extension of long-context capabilities for large language models (LLMs) in an efficient manner. The motivation behind the paper is to explore the potential of LLMs to extend their original context length beyond current limitations, with the goal of enhancing performance across various evaluation tasks while preserving capabilities over short contexts.
Invoking: `RetrieveInformation` with `{'query': 'claimed contributions and novelties of the paper', 'paper_title': 'Extending Llama-3’s Context'}`


The paper "Extending Llama-3’s Context Ten-Fold Overnight" claims to have extended the context length of Llama-3-8B-Instruct from 8K to 80K using QLoRA fine-tuning. The authors state that t

In [ ]:
print(output['output'])

I have created the content based on the questions and answers related to the paper "Extending Llama-3’s Context." The content has been saved as a markdown file with the title "Extending Llama-3’s Context.md" in the local directory.
